# Setup

In [1]:
path_q = 'qTable/77,8615.json'

# Learner

In [2]:
import random
import json

import dataclasses

@dataclasses.dataclass
class GameState:
    distance: tuple
    position: tuple
    surroundings: str
    food: tuple
    direction: int


class Learner(object):
    def __init__(self, display_width, display_height, block_size):
        # Game parameters
        self.display_width = display_width
        self.display_height = display_height
        self.block_size = block_size

        # Learning parameters
        self.epsilon = 0
        self.epsilon_decay = 0.9992
        self.lr = 0.03
        self.discount = .95

        # State/Action history
        self.qvalues = self.LoadQvalues()
        self.history = []

        # Action space
        self.actions = {
            0:'left',
            1:'right',
            2:'up',
            3:'down'
        }

    def Reset(self):
        self.history = []
        
    #def UploadQvalues(self, path"qvalues.json"):
        
    

    def LoadQvalues(self, path=path_q):
        with open(path, "r") as f:
            qvalues = json.load(f)
        return qvalues

    def SaveQvalues(self, path="qvalues.json"):
        with open(path, "w") as f:
            json.dump(self.qvalues, f)
            
    def act(self, dir, snake, food):
        state = self._GetState(dir, snake, food)

        # Epsilon greedy
        rand = random.uniform(0,1)
        if rand < self.epsilon:
            action_key = random.choices(list(self.actions.keys()))[0]
        else:
            state_scores = self.qvalues[self._GetStateStr(state)]
            action_key = state_scores.index(max(state_scores))
        action_val = self.actions[action_key]
        
        # Remember the actions it took at each state
        self.history.append({
            'state': state,
            'action': action_key
            })
        return action_val
    
    def UpdateQValues(self, reason):
        history = self.history[::-1]
        for i, h in enumerate(history[:-1]):
            if reason: # Snake Died -> Negative reward
                sN = history[0]['state']
                aN = history[0]['action']
                state_str = self._GetStateStr(sN)
                if reason == 'Steps': # Snake took too many steps
                    reward = -3
                else:  
                    reward = -30
                self.qvalues[state_str][aN] = (1-self.lr) * self.qvalues[state_str][aN] + self.lr * reward # Bellman equation - there is no future state since game is over
                reason = None
            else:
                s1 = h['state'] # current state
                s0 = history[i+1]['state'] # previous state
                a0 = history[i+1]['action'] # action taken at previous state
                
                x1 = s0.distance[0] # x distance at current state
                y1 = s0.distance[1] # y distance at current state
    
                x2 = s1.distance[0] # x distance at previous state
                y2 = s1.distance[1] # y distance at previous state
                
                if s0.food != s1.food: # Snake ate a food, positive reward
                    reward = 10
                elif (abs(x1) > abs(x2) or abs(y1) > abs(y2)): # Snake is closer to the food, positive reward
                    reward = 0.001
                else:
                    reward = -0.001 # Snake is further from the food, negative reward
                    
                state_str = self._GetStateStr(s0)
                new_state_str = self._GetStateStr(s1)
                self.qvalues[state_str][a0] = (1-self.lr) * (self.qvalues[state_str][a0]) + self.lr * (reward + self.discount*max(self.qvalues[new_state_str])) # Bellman equation


    def _GetState(self, dir, snake, food):
        snake_head = snake[-1]
        dist_x = food[0] - snake_head[0]
        dist_y = food[1] - snake_head[1]

        if dist_x > 0:
            pos_x = '1' # Food is to the right of the snake
        elif dist_x < 0:
            pos_x = '0' # Food is to the left of the snake
        else:
            pos_x = 'NA' # Food and snake are on the same X file

        if dist_y > 0:
            pos_y = '3' # Food is below snake
        elif dist_y < 0:
            pos_y = '2' # Food is above snake
        else:
            pos_y = 'NA' # Food and snake are on the same Y file

        sqs = [
            (snake_head[0]-self.block_size, snake_head[1]),   
            (snake_head[0]+self.block_size, snake_head[1]),         
            (snake_head[0],                  snake_head[1]-self.block_size),
            (snake_head[0],                  snake_head[1]+self.block_size),
        ]
        
        surrounding_list = []
        for sq in sqs:
            if sq[0] < 0 or sq[1] < 0: # off screen left or top
                surrounding_list.append('1')
            elif sq[0] >= self.display_width or sq[1] >= self.display_height: # off screen right or bottom
                surrounding_list.append('1')
            elif sq in snake[:-1]: # part of tail
                surrounding_list.append('1')
            else:
                surrounding_list.append('0')
        surroundings = ''.join(surrounding_list)

        return GameState((dist_x, dist_y), (pos_x, pos_y), surroundings, food, dir)

    def _GetStateStr(self, state):
        return str((state.position[0],state.position[1],state.surroundings, state.direction))

In [3]:
#%%
import pygame
import random
#import Learner

pygame.init()

#%% CONSTANTS
YELLOW = (255, 255, 102)
BLACK = (0, 0, 0)
GREEN = (0, 255, 0)
BLUE = (50, 153, 213)

BLOCK_SIZE = 10 
DIS_WIDTH = 600
DIS_HEIGHT = 400

QVALUES_N = 100
FRAMESPEED = 500

#%% Game 

def GameLoop():
    global dis
    
    dis = pygame.display.set_mode((DIS_WIDTH, DIS_HEIGHT))
    pygame.display.set_caption('Snake')
    clock = pygame.time.Clock()

    # Starting position of snake
    x1 = DIS_WIDTH / 2
    y1 = DIS_HEIGHT / 2
    x1_change = 0
    y1_change = 0
    snake_list = [(x1,y1)]
    length_of_snake = 1

    # Create first food
    foodx = round(random.randrange(0, DIS_WIDTH - BLOCK_SIZE) / 10.0) * 10.0
    foody = round(random.randrange(0, DIS_HEIGHT - BLOCK_SIZE) / 10.0) * 10.0

    dead = False
    reason = None

    direction = 0
    steps = 0
    while not dead:
        # Get action from agent
        action = learner.act(direction, snake_list, (foodx,foody))
        steps +=1
        if action == "left":
            x1_change = -BLOCK_SIZE
            y1_change = 0
            direction = 0
        elif action == "right":
            x1_change = BLOCK_SIZE
            y1_change = 0
            direction = 1
        elif action == "up":
            y1_change = -BLOCK_SIZE
            x1_change = 0
            direction = 2
        elif action == "down":
            y1_change = BLOCK_SIZE
            x1_change = 0
            direction = 3

        # Move snake
        x1 += x1_change
        y1 += y1_change
        snake_head = (x1,y1)
        snake_list.append(snake_head)

        # Check if snake is off screen
        if x1 >= DIS_WIDTH or x1 < 0 or y1 >= DIS_HEIGHT or y1 < 0:
            reason = 'Screen'
            dead = True

        # Check if snake hit tail
        if snake_head in snake_list[:-1]:
            reason = 'Tail'
            dead = True

        # Check if snake ate food
        if x1 == foodx and y1 == foody:
            foodx = round(random.randrange(0, DIS_WIDTH - BLOCK_SIZE) / 10.0) * 10.0
            foody = round(random.randrange(0, DIS_HEIGHT - BLOCK_SIZE) / 10.0) * 10.0
            length_of_snake += 1
            steps = 0
        
        if steps > 1000:
            reason = 'Steps'
            dead = True

        # Delete the last cell since we just added a head for moving, unless we ate a food
        if len(snake_list) > length_of_snake:
            del snake_list[0]

        # Draw food, snake and update score
        dis.fill(BLUE)
        DrawFood(foodx, foody)
        DrawSnake(snake_list)
        DrawScore(length_of_snake - 1)
        pygame.display.update()

        
        # Next Frame
        clock.tick(FRAMESPEED)

    return length_of_snake - 1, reason

def DrawFood(foodx, foody):
    pygame.draw.rect(dis, GREEN, [foodx, foody, BLOCK_SIZE, BLOCK_SIZE])   

def DrawScore(score):
    font = pygame.font.SysFont("comicsansms", 35)
    value = font.render(f"Score: {score}", True, YELLOW)
    dis.blit(value, [0, 0])

def DrawSnake(snake_list):
    for x in snake_list:
        pygame.draw.rect(dis, BLACK, [x[0], x[1], BLOCK_SIZE, BLOCK_SIZE])



#%%
game_count = 1

learner = Learner(DIS_WIDTH, DIS_HEIGHT, BLOCK_SIZE)

high_score = 0
while True:
    learner.Reset()
    learner.epsilon *= learner.epsilon_decay # Decay epsilon
    score, reason = GameLoop()
    print(f"Games: {game_count}; Score: {score}; Reason: {reason};  Current high score: {high_score}") # Output results of each game to console to monitor as agent is training
    if score > high_score:
        high_score = score
    game_count += 1

pygame 2.5.2 (SDL 2.28.3, Python 3.10.7)
Hello from the pygame community. https://www.pygame.org/contribute.html
Games: 1; Score: 35; Reason: Tail;  Current high score: 0
Games: 2; Score: 48; Reason: Tail;  Current high score: 35
Games: 3; Score: 70; Reason: Tail;  Current high score: 48
Games: 4; Score: 64; Reason: Tail;  Current high score: 70
Games: 5; Score: 44; Reason: Tail;  Current high score: 70
Games: 6; Score: 22; Reason: Tail;  Current high score: 70
Games: 7; Score: 28; Reason: Tail;  Current high score: 70
Games: 8; Score: 53; Reason: Screen;  Current high score: 70
Games: 9; Score: 53; Reason: Tail;  Current high score: 70
Games: 10; Score: 0; Reason: Steps;  Current high score: 70
Games: 11; Score: 0; Reason: Steps;  Current high score: 70
Games: 12; Score: 21; Reason: Tail;  Current high score: 70
Games: 13; Score: 57; Reason: Screen;  Current high score: 70
Games: 14; Score: 35; Reason: Screen;  Current high score: 70
Games: 15; Score: 0; Reason: Steps;  Current high s

KeyboardInterrupt: 